In [2]:
#Libraries
import numpy as np
import scipy.integrate

#Plotting & animation
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FFMpegWriter

#Astro
import astropy.constants as const
import astropy.units as u

#GUI
import ipywidgets as widgets

# %matplotlib osx 
# ^ UNCOMMENT THIS LINE IF USING MAC

%matplotlib qt 
# ^ UNCOMMENT THIS LINE IS USING WINDOWS

### Class Design Time! (Revisions probably needed) (Classes are in progress below!!)

Things we will need to represent:
- Planets
- Stars
- Solar system (which contains a list of planets, as well as a star? Maybe that's the way to do it. Would make potential future expansion easier if we ever want to add binary system/etc/other system with multiple stars)
- Transits (should just be a list representing flux over a time interval)
- Vectors (data abstraction -> make a class!)
- User inputs (keep this organized though)
- GUI: Widgets seems like possible solution? Tutorial: https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916

- How to handle time? (By days? Can users speed up/slow it down?)

In [3]:
class Vector: 
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __str__(self):
        return f'<{self.x}, {self.y}>'
    
    def length(self):
        return math.sqrt(pow(self.x, 2) + pow(self.y, 2))
    
    def angle(self):
        return math.atan2(self.y, self.x)

In [38]:
# Physics stuff. Functions for useful equations go here

# '''
# Differential equation for finding orbital velocity & acceleration for Planet orbiting Star at some point in time, T
# '''
# def gravitation(planet, star, t):
#     pos = planet.position
#     vel = [planet.velocity.x, planet.velocity.y]
    
#     pos_12 = np.linalg.norm(pos)
#     dv_dt = star.mass * (-pos) / pow(pos_12, 3)
#     dpos_dt = vel
    
#     return dpos_dt.extend(dv_dt)

'''
Represents the differential equation for the law of gravitation.
rv: an array containing the positions and velocities of 2 objects
t: time
'''
def gravitation(rv, t, mass_star, mass_planet):
    r1 = rv[:2] #* u.au
    v1 = rv[2:4] #* (u.meter/u.second)
    r12 = np.linalg.norm(r1)
    
    dv1bydt = mass_planet * (-r1)/(pow(r12, 3))
    dr1bydt = v1
    
    derivatives = np.concatenate((dr1bydt, dv1bydt))
    return derivatives

Methods/attributes a planet should have:


In [106]:
class Planet:
    
    '''
    radius: radius of planet in Earth radii
    mass: mass of planet in Earth masses
    distance: planet's distance from star (au) [distance_x, distance_y]
    velocity: planet's velocity (m/s) [vx, vy]
    '''
#     def __init__(self, radius, mass, position, velocity, accel=None, name=None):
#         self.radius = radius #* u.R_earth
#         self.mass = mass #* u.M_earth
#         self.position = position #* u.au #todo: figure this out
#         self.velocity = velocity #todo: figure this out
#         self.acceleration = accel #todo: figure this out
#         self.name = name
    def __init__(self, mass, radius, distance, init_velocity, color, implemented=True):
        self.mass = mass #* u.M_earth
        self.distance = distance #* u.au
        self.velocity = init_velocity #* (u.meter/u.second)
        self.implemented = implemented
        self.color = color
        self.radius = radius
        
        
    def __repr__(self):
        #todo: print representation
        return
        
#     def __str__(self):
#         return f'Planet {self.name}. Radius={self.radius}, mass={self.mass}'
    
    def set_mass(self, mass):
        self.mass = mass
    
    def set_distance(self, distance):
        self.distance = distance
    
    def set_implemented(self, status):
        self.implemented = status
    

Methods/attributes a star should have:


In [107]:
class Star:
    
    '''
    mass: mass of star in solar masses
    position: list representing x, y position of star (defaults to origin)
    velocity: vector representing x and y components of star's velocity (defaults to 0; ie. stationary)
    radius is in Earth radii
    '''
    def __init__(self, mass, radius, position=[0,0], velocity=[0, 0], name='Sol'):
        self.mass = mass #* u.M_sun
        self.position = position #* u.au
        self.velocity = velocity #* (u.meter/u.second)
        self.name = name
        self.radius = radius
    
    def __repr__(self):
        #todo: print representation
        return
        
#     def __str__(self):
#         return f'Star {self.name}. Radius={self.radius}, mass={self.mass}'   

Methods/attributes a solar system should have:


In [142]:
class SolarSystem: 
    
    def __init__(self, star, planets=[]):
        self.star = star
        self.planets = planets
    
    def get_planet_masses(self):
        masses = []
        for planet in self.planets:
            if planet.implemented:
                masses.append(planet.mass)
        return masses
    
    def get_planet_distances(self):
        distances = []
        for planet in self.planets:
            if planet.implemented:
                distances.append(planet.distance[0])
        return distances
    
    def most_massive_planet(self):
        masses = self.get_planet_masses()
        largest = max(masses)
        for planet in self.planets:
            if planet.implemented and planet.mass == largest:
                return planet
        return None
    
    def farthest_planet(self):
        distances = self.get_planet_distances()
        farthest = max(distances)
        for planet in self.planets:
            if planet.implemented and planet.distance[0] == farthest:
                return planet
        return None
            
        

In [160]:
p1 = Planet(3 * 10**(-5), 1.0, [1, 0], [0, 0.75], color='blue')

In [161]:
p2 = Planet(5 * 10**(-5), 3.0, [2, 0], [0, 0.75], color='green')

In [162]:
p3 = Planet(7 * 10**(-5), 6.0, [2.5, 0], [0, 0.75], color='purple')

In [163]:
p4 = Planet(10 * 10**(-5), 8.0, [3, 0], [0, 0.75], color='cyan')

In [164]:
p5 = Planet(13 * 10**(-5), 10.0, [5, 0], [0, 0.75], color='red', implemented=False)

In [23]:
# j = Planet(11.2, 317.8, name='jupiter')
# j.mass

TypeError: __init__() missing 2 required positional arguments: 'position' and 'velocity'

In [165]:
sun = Star(1, 100.0)

In [57]:
# # Package initial parameters into one array (just easier to work with this way)
# r1 = earth.distance
# v1 = earth.velocity
# init_params = np.array([r1, v1])
# init_params = init_params.flatten()
# time_span = np.linspace(0, 100, 500)  # run for t=5 (500 points)
# # Run the ODE solver
# sol = scipy.integrate.odeint(gravitation, init_params, time_span, args=(earth.mass, sun.mass))

In [58]:
# # Package initial parameters into one array (just easier to work with this way)
# r2 = earth2.distance
# v2 = earth2.velocity
# init_params = np.array([r2, v2])
# init_params = init_params.flatten()
# time_span = np.linspace(0, 100, 500)  # run for t=5 (500 points)
# # Run the ODE solver
# sol2 = scipy.integrate.odeint(gravitation, init_params, time_span, args=(earth2.mass, sun.mass))

In [59]:
# # Package initial parameters into one array (just easier to work with this way)
# r3 = earth3.distance
# v3 = earth3.velocity
# init_params = np.array([r3, v3])
# init_params = init_params.flatten()
# time_span = np.linspace(0, 100, 500)  # run for t=5 (500 points)
# # Run the ODE solver
# sol3 = scipy.integrate.odeint(gravitation, init_params, time_span, args=(earth3.mass, sun.mass))

In [60]:
# r1_sol = sol[:, :2]
# r2_sol = sol2[:, :2]
# r3_sol = sol3[:, :2]

In [166]:
#Set up solar system object
test_system = SolarSystem(sun, [p1, p2, p3, p4, p5])

In [173]:
# Set up & solve ODE for motion of each planet in the solar system
# todo: automatically set time to run the animation on (determine params for timespan)
# todo: set plot dimensions automatically

all_orbit_solutions = []
# time to run simulation should depend on how far the farthest planet is
time_span = np.linspace(0, max(test_system.get_planet_distances()) * 100, 1000) 
axes_limits = [-max(test_system.get_planet_distances()) * 10, max(test_system.get_planet_distances()) * 5]
lightcurve = []

for planet in test_system.planets:
    if planet.implemented:
        init_params = np.array([planet.distance, planet.velocity])
        init_params = init_params.flatten()
        sol = scipy.integrate.odeint(gravitation, init_params, time_span, args=(planet.mass, test_system.star.mass))
        sol_for_planet = sol[:, :2]
        all_orbit_solutions.append(sol_for_planet)

In [174]:
# Initialize writer 
metadata = dict(title='Orbit Test', artist='Matplotlib')
writer = FFMpegWriter(fps=50, metadata=metadata, bitrate=200000) # change fps for different frame rates
fig = plt.figure(dpi=200)

In [175]:
# SAVE AS MP4 (will be saved in whatever directory you are working in)
fig, (ax1, ax2) = plt.subplots(2, 1)

with writer.saving(fig, "orbit_test_14.mp4", dpi=200):
    for i in range(len(time_span)):

        # Animation of orbiting planets
        ax1.clear()
        
        for planet_idx in range(len(test_system.planets)):
            if test_system.planets[planet_idx].implemented:
                planet_sols = all_orbit_solutions[planet_idx]
                ax1.plot(planet_sols[:i, 0], planet_sols[:i, 1], color = test_system.planets[planet_idx].color, alpha=0.5) # path
                ax1.scatter(planet_sols[i,0], planet_sols[i,1],color = test_system.planets[planet_idx].color, marker="o",s=20, zorder=5) # planet

        
        ax1.scatter(0, 0, color="orange",marker="*", s=50, zorder=5) # star
        
        ax1.set_xlim(axes_limits[0], axes_limits[1])
        ax1.set_ylim(axes_limits[0], axes_limits[1])
        ax1.set_title('Solar System Animation')


        
        # TODO: Animation of lightcurve w/ transits
        ax2.clear()
        
        # 1. define observation of transit to be when planet moves back to where it started (position = [orig_distance, 0]) (+/- a bit)
        # 2. lightcurve value is 1.0 if no transits
        # 3. lightcurve value is 1.0 - (planet size/star ratio) (placeholder: planet mass * 100 for test)
        
        flux_level = 1.0
        for planet_idx in range(len(test_system.planets)):
            if test_system.planets[planet_idx].implemented:
                planet_sols = all_orbit_solutions[planet_idx]
                dist = test_system.planets[planet_idx].distance
                loc = [planet_sols[i, 0], planet_sols[i, 1]]
                if abs(loc[0] - dist[0]) < 0.1 and abs(loc[1]) < 0.1:
                #if np.absolute(np.array([planet_sols[:i, 0], planet_sols[:i, 1]]) - np.array(test_system.planets[planet_idx].distance) < np.array([0.1, 0.1]):
                    flux_level -= pow((test_system.planets[planet_idx].radius/test_system.star.radius), 2)
            lightcurve.append(flux_level)
            
        ax2.plot(lightcurve[:i], color = 'black', alpha=0.6)
        ax2.scatter(i, flux_level, color = 'orange', marker = 'o', s=10, zorder = 5)
        
        ax2.set_xlim(0, len(time_span))
        ax2.set_ylim(0.98, 1.005)
        ax2.set_title('Lightcurve')
        
        plt.draw()
        plt.pause(0.01)
        writer.grab_frame()